# Feature Engineering

Hemos estudiamos las ideas fundamentales de machine learning, pero todos los ejemplos vistos asumieron que teníamos datos numéricos en un formato [n_samples, m_features].

En aplicaciones reales, los datos rara vez vienen ordenados de esa manera. Con esto en mente, uno de los pasos más importantes para usar machine learning en la práctica es la **ingeniería de features o feature engineering**, esto es, tomar cualquier información que exista sobre el problema a resolver y convertirla en números que puedan usarse para construir una matriz de features.

Normalmente este proceso es conocido como **vectorización**, ya que involucra la idea de convertir datos arbitrarios en vectores bien formados.

## Trabajando con features categóricas

Imaginemos que estamos explorando ciertos datos de precios de propiedades y, junto a features numéricas como el precio y el total de habitaciones, también tenemos información sobre el barrio en que se encuentra cada propiedad. Por ejemplo, los datos podrían verse así:

In [ ]:
data = [{'precio': 850000, 'habitaciones': 4, 'barrio': 'Palermo'},
        {'precio': 700000, 'habitaciones': 3, 'barrio': 'San Telmo'},
        {'precio': 650000, 'habitaciones': 3, 'barrio': 'Villa Luro'},
        {'precio': 600000, 'habitaciones': 2, 'barrio': 'La Boca'}]

Usaremos la técnica **one-hot encoding**, que crea columnas extra indicando la presencia o ausencia de una categoría con un valor de 1 o 0, respectivamente. Cuando los datos vienen como una lista de diccionarios, la clase **DictVectorizer** hace esto automáticamente:

In [ ]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse = False, dtype = int)
vec.fit_transform(data)

Con las features categóricas codificadas de esta manera, podemos proceder como de costumbre para ajustar un modelo con Scikit-Learn.

Para ver el significado de cada columna, podemos inspeccionar los nombres de las features:

In [ ]:
vec.get_feature_names()

Hay una clara desventaja en esta aproximación: si cada categoría tiene muchos valores posibles, esto puede incrementar en gran medida el tamaño del dataset. Sin embargo, como los datos codificados contienen principalmente ceros, una **representación dispersa (sparse matrix)** podría ser una solución eficiente:

In [ ]:
vec = DictVectorizer(sparse = True, dtype = int)
vec.fit_transform(data)

Notemos que la matriz dispersa almacena **únicamente los valores distintos de cero**: en nuestro caso, de un total de 24 elementos (4 x 6), sólo almacenó 12. Esto hace mucho más eficiente el procesamiento. Muchos (aunque no todos) de los estimadores de Scikit-Learn aceptan representaciones dispersas cuando se ajustan y evalúan los modelos.

Alternativamente, **sklearn.preprocessing.OneHotEncoder y sklearn.feature_extraction.FeatureHasher** son dos funcionalidades adicionales que Scikit-Learn incluye para soportar este tipo de codificación.

## Trabajando con texto

Otra necesidad común en feature engineering es convertir texto a un conjunto de valores numéricos representativos. Por ejemplo, la mayoría del mining automático de datos de redes sociales se basa en alguna forma de codificación del texto como números. Uno de los métodos más simples es codificar los datos por medio del  conteo de palabras (*word counts*), que consiste en tomar cada fragmento de texto, contar las ocurrencias de cada palabra en él y volcar los resultados en una tabla.

Por ejemplo, consideremos el siguiente dataset de tres frases:

In [ ]:
textos = ['científico de datos',
          'datos estructurados',
          'pensamiento científico']

Para vectorizar este dataset basado en el conteo de palabras, podríamos construir una columna representando cada palabra: "científico", "datos", "pensamiento", etc.

Para esto usamos **CountVectorizer**:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(textos)
X

Podemos visualizar el output del CountVectorizer utilizando el método `todense()`, que convierte la **matrix dispersa a una matriz densa**.

In [ ]:
X.todense()

Podemos volcar este resultado en un `DataFrame`. Para obtener el encabezado de las columnas, vamos a utilizar el método `get_feature_names()`, propio del vectorizador.

In [ ]:
import pandas as pd
pd.DataFrame(data = X.todense(), columns = vec.get_feature_names())

**Term frequency-inverse document frequency (TF–IDF)** es una técnica alternativa que computa la frecuencia relativa de cada palabra por documento, ponderada por la inversa de su frecuencia relativa a lo largo del *corpus* (colección de documentos). Este método funciona mejor con ciertos algoritmos de clasificación.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
X = vec.fit_transform(textos)
pd.DataFrame(data = X.todense(), columns = vec.get_feature_names())

## Features derivadas

Consideremos el siguiente dataset:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

x = np.array([1, 2, 3, 4, 5])
y = np.array([4, 2, 1, 3, 7])
plt.scatter(x, y);

Este dataset claramente no puede ser descrito correctamente por una línea recta. Sin embargo, podemos ajustar una línea a los datos usando `LinearRegression` y obtener el siguiente resultado:

In [ ]:
from sklearn.linear_model import LinearRegression
X = x[:, np.newaxis]
model = LinearRegression().fit(X, y)
yfit = model.predict(X)
plt.scatter(x, y)
plt.plot(x, yfit);

Para poder explicar correctamente estos datos, necesitaríamos un modelo más sofisticado que describa la relación entre x e y. 

Una aproximación a esto es transformar los datos, agregando columnas extra de features para agregar más flexibilidad al modelo. Por ejemplo, podemos agregar **features polinómicas** a los datos de esta forma:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 3, include_bias = False)
X2 = poly.fit_transform(X)
print(X2)

La matriz de features derivada tiene una columna representando x, una segunda columna representando x al cuadrado, y una tercer columna representando x al cubo. Computar una regresión lineal en esta entrada expandida nos da un ajuste mucho más cercano a nuestros datos:

In [ ]:
model = LinearRegression().fit(X2, y)
yfit = model.predict(X2)
plt.scatter(x, y)
plt.plot(x, yfit);

La idea de mejorar una solución, no cambiando el modelo, pero tranformando la entrada, es fundamental para muchos de los métodos más poderosos de machine learning.   

Más generalmente, éeste es un tema que motiva la creación de las técnicas conocidas como kernel methods.

## Trabajando con datos faltantes

Otra necesidad común en feature engineering es el manejo de datos faltantes. Por ejemplo, podríamos tener un dataset como este:

In [ ]:
from numpy import nan

X = np.array([[ nan, 0,   3  ],
              [ 3,   7,   9  ],
              [ 3,   5,   2  ],
              [ 4,   nan, 6  ],
              [ 8,   8,   1  ]])

y = np.array([14, 16, -1,  8, -5])

Para aplicar un modelo de machine learning a estos datos, primero debemos reemplazar los datos faltantes con algún valor apropiado. Esto se conoce como **imputación de datos faltantes** y las estrategias van desde las simples (reemplazar valores faltantes con la media de la columna) hasta las más sofisticadas (como usar modelos robustos para imputación).

Como un ejemplo simple de imputación, usaremos la media. Scikit-Learn provee la clase **Imputer**:

In [ ]:
from sklearn.preprocessing import Imputer
imp = Imputer(strategy = 'mean')
X2 = imp.fit_transform(X)
X2

## Pipelines

Si queremos armar una secuencia de transformaciones, puede ser tedioso hacerlo a mano. Por ejemplo, podríamos querer hacer algo como esto:

1. Imputar valores perdidos usando la media
2. Transformar features a cuadráticas
3. Ajustar una regresión lineal

Para organizar este tipo de pipeline de procesamiento, Scikit-Learn provee la clase **Pipeline**:

In [ ]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(Imputer(strategy = 'mean'),
                      PolynomialFeatures(degree = 2),
                      LinearRegression())

Este **Pipeline** se comporta como un objeto estándar de Scikit-Learn, y **aplicará todos los pasos especificados a cualquier dato de entrada**:

In [ ]:
model.fit(X, y)  # X contiene valores faltantes, pero el primer paso del Pipeline es imputarles la media
print(y)
print(model.predict(X))